In [1]:
import warnings
warnings.simplefilter('ignore')
from sklearn.tree import DecisionTreeClassifier
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.externals import joblib



class PandasDummies(BaseEstimator, TransformerMixin):
    def transform(self, X, *_):
        return pd.get_dummies(X)
    
    def fit(self, *_):
        return self

In [2]:
teamData=pd.read_csv("../data/clean_team_stats.csv")
teamData.columns



Index(['team_no', 'variable', 'first_downs', 'first_downs_by_penalty',
       'third_down_percentage', 'fourth_down_percentage',
       'average_interception_yards', 'average_kickoff_return_yards',
       'average_punt_return_yards', 'interceptions', 'net_average_punt_yards',
       'net_passing_yards', 'net_passing_yards_per_game',
       'passing_first_downs', 'passing_touchdowns', 'rushing_first_downs',
       'rushing_attempts', 'rushing_touchdowns', 'rushing_yards',
       'rushing_yards_per_game', 'total_offensive_plays', 'total_points',
       'total_points_per_game', 'total_touchdowns', 'total_offensive_yards',
       'total_penalties_yards', 'yards_per_game', 'yards_per_pass_attempt',
       'yards_per_rush_attempt', 'total_penalties', 'total_yards_penalized',
       'total_defensive_sacks', 'yards_lost_from_sacks', 'total_punts_kicked',
       'total_punt_yards', 'total_team_penalties', 'abbr'],
      dtype='object')

In [3]:
X = teamData[[  'third_down_percentage',
 'yards_per_pass_attempt',
 'passing_touchdowns',
  'rushing_touchdowns',
'total_offensive_plays'
 
 
       ]]

y= teamData['total_points'].values.reshape(-1, 1)

In [4]:
X

,third_down_percentage,yards_per_pass_attempt,passing_touchdowns,rushing_touchdowns,total_offensive_plays
0,42.99,6.8,19,30,1028
1,39.33,7.7,22,12,917
2,34.84,7.8,19,20,815
3,45.88,9.5,33,32,1019
4,46.20,11.9,10,52,894
5,43.87,8.0,35,35,1054
6,38.98,6.2,18,15,818
7,37.81,7.0,26,16,972
8,38.46,8.0,19,33,976
9,42.51,7.6,20,18,986


In [5]:
def linerregressionRun(X, y, times):
    
    MSEresults=[]
    R2results=[]
    rffit=[]
    rfscore=[]
    i=0
    rfFeatures=[]
    steps=[]
   
    while i<times:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)
        model = make_pipeline(PandasDummies(), StandardScaler(), LinearRegression())
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        MSE = mean_squared_error(y_test, predictions)
        r2 = model.score(X_test, y_test)
        print(f"MSE: {MSE}, R2: {r2}")
        MSEresults.append(MSE)
        R2results.append(r2)
        joblib.dump(model, F"{i}my_model.pkl")
        rf = RandomForestRegressor(n_estimators=200, max_leaf_nodes=5)
        rf = rf.fit(X_train, y_train)
        acc = rf.score(X_test, y_test)
        rfF=sorted(zip(rf.feature_importances_, X.columns), reverse=True)
        rfFeatures.append(rfF)
        steps.append(model.steps)
        rffit.append(rf)
        
        
        rfscore.append(acc)
        print(f"accuracy = {acc}")
        i=i+1
    results={"MSE":MSEresults, 
             "r2":R2results,
             "RandomForest":rffit,
             "RFScore":rfscore,
             "features":rfFeatures,
             "steps":steps
             }
    return pd.DataFrame(results)
    


In [6]:
TUNNING = linerregressionRun(X, y, 5)

MSE: 458.15130798405755, R2: 0.9570546121569234
accuracy = 0.8098560657875694
MSE: 458.15130798405755, R2: 0.9570546121569234
accuracy = 0.8078003852099798
MSE: 458.15130798405755, R2: 0.9570546121569234
accuracy = 0.8070102282000247
MSE: 458.15130798405755, R2: 0.9570546121569234
accuracy = 0.8063072654902562
MSE: 458.15130798405755, R2: 0.9570546121569234
accuracy = 0.795412461887079


In [7]:
TUNNING.columns

Index(['MSE', 'r2', 'RandomForest', 'RFScore', 'features', 'steps'], dtype='object')

In [8]:
TUNNING["features"][0]

[(0.32233360620444323, 'third_down_percentage'),
 (0.21060709428755872, 'passing_touchdowns'),
 (0.19566096920317208, 'yards_per_pass_attempt'),
 (0.1415632834163407, 'rushing_touchdowns'),
 (0.12983504688848524, 'total_offensive_plays')]

In [9]:
TUNNING["RandomForest"][0].base_estimator_


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [10]:
X2 = teamData[[   'third_down_percentage',
 'yards_per_pass_attempt',
 'passing_touchdowns',
  'rushing_touchdowns',
'total_offensive_plays',
 ]]

y2= teamData['total_points'].values.reshape(-1, 1)
              

In [11]:


some_totally_random_model = joblib.load("0my_model.pkl")

In [16]:
X2

,third_down_percentage,yards_per_pass_attempt,passing_touchdowns,rushing_touchdowns,total_offensive_plays
0,42.99,6.8,19,30,1028
1,39.33,7.7,22,12,917
2,34.84,7.8,19,20,815
3,45.88,9.5,33,32,1019
4,46.20,11.9,10,52,894
5,43.87,8.0,35,35,1054
6,38.98,6.2,18,15,818
7,37.81,7.0,26,16,972
8,38.46,8.0,19,33,976
9,42.51,7.6,20,18,986


In [17]:

X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.40, random_state=42)

In [18]:
results=some_totally_random_model.predict(X2)


In [19]:
results


array([[419.68146028],
       [312.1853118 ],
       [324.15424464],
       [534.69564148],
       [508.70702099],
       [561.71385599],
       [279.41436224],
       [366.29723676],
       [434.80326935],
       [349.33726071],
       [564.10353285],
       [246.54060188],
       [402.81311312],
       [636.19106549],
       [294.64220527],
       [406.46449213],
       [218.81910303],
       [456.79070428],
       [357.42523124],
       [259.43830732],
       [443.47088485],
       [279.98200179],
       [346.03550503],
       [431.08492019],
       [389.85295692],
       [437.49766809],
       [480.91990924],
       [420.36398465],
       [293.81269358],
       [348.22250918],
       [591.54095289],
       [418.77133127],
       [333.46000455],
       [477.64413704],
       [377.25306199],
       [247.95001027],
       [324.83463731],
       [413.49288859],
       [298.11898173],
       [310.86021504],
       [281.00640186],
       [420.95733331],
       [466.00642249],
       [340

In [20]:
teamData['total_points'][60]

324

In [26]:
def mlyards(one, two, three, four, five):
    #create df with these 5 variables with column names from model
    data={'third_down_percentage': one,
        'yards_per_pass_attempt':two,
        'passing_touchdowns':three,
        'rushing_touchdowns':four,
        'rushing_yards_per_game':five,
         }
    df=pd.DataFrame(data, index=[0])
    #import model
    some_totally_random_model = joblib.load("0my_model.pkl")
    #run number with model
    results=some_totally_random_model.predict(df)
    #return result as int
    return results

In [28]:
mlyards(42.99,6.8,19,30,1028)

array([[419.68146028]])